In [4]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt


from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import PoissonRegressor
from sklearn.ensemble import RandomForestRegressor





**Table of contents**<a id='toc0_'></a>    
- [Creating a prediction model using Poisson Regression Model](#toc1_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Creating a prediction model using Poisson Regression Model](#toc0_)

In [ ]:
city_comp_df = pd.read_parquet('..\\data\\feature_store\\city_comp_df.parquet')

In [ ]:


# Ensure that categorical variables are properly specified
categorical_vars = [ 'age_cat', 'gender', 'max_bmi_cat', 'max_major_city']
for var in categorical_vars:
    city_comp_df[var] = city_comp_df[var].astype('category')

reference_city = 'Riyadh'  # e.g., 'New York' or whichever city you prefer

# ----------------------------------------------------------
# 1. Split the data into training and testing sets.
# ----------------------------------------------------------
# For reproducibility, set a random_state (e.g., 42)

train_data, test_data = train_test_split(city_comp_df, test_size=0.2, random_state=42)

# ----------------------------------------------------------
# 2. Fit a Poisson regression model on the training data.
# ----------------------------------------------------------
# Use the same formula as in your robust model (without the clustering for prediction purposes).
formula_full_with_policy = (
    'total_complications ~ C(max_major_city, Treatment(reference="%s"))  + age + C(gender) + max_bmi + total_comorbidities + total_dm_icd'
    % reference_city
)

# Fit the model on the training data.
poisson_cluster_model = smf.glm(
    formula=formula_full_with_policy,
    data=city_comp_df,
    family=smf.families.Poisson()
).fit(cov_type='cluster', cov_kwds={'groups': city_comp_df['policy_number']})

# Optionally, view the training model's summary:
print(poisson_cluster_model.summary())

# ----------------------------------------------------------
# 3. Use the fitted model to predict complication counts on the test data.
# ----------------------------------------------------------
test_data = test_data.copy()  # To avoid SettingWithCopyWarning
test_data['predicted_complications'] = poisson_cluster_model.predict(test_data)

# ----------------------------------------------------------
# 4. Evaluate the prediction accuracy.
# ----------------------------------------------------------
# Calculate common metrics: Mean Squared Error (MSE) and Mean Absolute Error (MAE)
mse = mean_squared_error(test_data['total_complications'], test_data['predicted_complications'])
mae = mean_absolute_error(test_data['total_complications'], test_data['predicted_complications'])

print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)

# ----------------------------------------------------------
# 5. Visualize Actual vs. Predicted complication counts.
# ----------------------------------------------------------
plt.figure(figsize=(8, 6))
plt.scatter(test_data['total_complications'], test_data['predicted_complications'], alpha=0.6)
plt.plot([test_data['total_complications'].min(), test_data['total_complications'].max()],
         [test_data['total_complications'].min(), test_data['total_complications'].max()],
         color='red', linestyle='--', label='Ideal Fit')
plt.xlabel("Actual Number of Complications")
plt.ylabel("Predicted Number of Complications")
plt.title("Actual vs. Predicted Complication Counts")
plt.legend()
plt.show()



In [ ]:



# -------------------------------------------------------------------
# 1. Prepare the data for prediction modeling using scikit-learn.
# -------------------------------------------------------------------
# Select predictors and the outcome.
predictors = ['max_major_city', 'age', 'gender', 'max_bmi', 'total_comorbidities']
y = city_comp_df['total_complications']
X = city_comp_df[predictors]

# Convert categorical variables to dummy/indicator variables.
# For max_major_city, set "Riyadh" as the baseline by dropping the first dummy.
X_encoded = pd.get_dummies(X, columns=['max_major_city', 'gender'], drop_first=True)

# -------------------------------------------------------------------
# 2. Split the data into training and testing sets.
# -------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# -------------------------------------------------------------------
# 3. Create and train the Poisson regression prediction model.
# -------------------------------------------------------------------
# Note: PoissonRegressor is available in scikit-learn (v0.23+).
# alpha=0.0 disables regularization, and max_iter ensures convergence.
poisson_reg = PoissonRegressor(alpha=0.0, max_iter=1000)
poisson_reg.fit(X_train, y_train)

# -------------------------------------------------------------------
# 4. Predict the number of complications on the test set.
# -------------------------------------------------------------------
y_pred = poisson_reg.predict(X_test)

# -------------------------------------------------------------------
# 5. Evaluate the prediction accuracy.
# -------------------------------------------------------------------
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)



# -------------------------------------------------------------
# 3. Random Forest Regressor
# -------------------------------------------------------------
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# Evaluate Random Forest predictions.
mse_rf = mean_squared_error(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)


# -------------------------------------------------------------
# 4. XGBoost Regressor
# -------------------------------------------------------------
xgb = XGBRegressor(n_estimators=100, random_state=42, objective='reg:squarederror')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

# Evaluate XGBoost predictions.
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)


# -------------------------------------------------------------
# 5. Print the prediction results.
# -------------------------------------------------------------

# Create a dictionary mapping each model's name to its corresponding MSE and MAE.
results = {
    "PoissonRegressor": {"MSE": mse, "MAE": mae},
    "Random Forest Regressor": {"MSE": mse_rf, "MAE": mae_rf},
    "XGBoost Regressor": {"MSE": mse_xgb, "MAE": mae_xgb},
}

# Loop through the dictionary and print the results in an efficient manner.
for model_name, metrics in results.items():
    print(f"{model_name} Prediction Results:")
    print(f"Mean Squared Error (MSE): {metrics['MSE']:.3f}")
    print(f"Mean Absolute Error (MAE): {metrics['MAE']:.3f}\n")

# -------------------------------------------------------------
# 6. Plot the prediction vs actual results.
# -------------------------------------------------------------


# Define the predictions and titles in a list for efficient plotting.
predictions = [
    (y_pred, "Poisson: Actual vs. Predicted"),
    (y_pred_rf, "Random Forest: Actual vs. Predicted"),
    (y_pred_xgb, "XGBoost: Actual vs. Predicted")
]

# Determine the range for the ideal fit line.
x_min, x_max = y_test.min(), y_test.max()

# Create a figure with 1 row and 3 columns.
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Loop over the axes and corresponding prediction data.
for ax, (y_pred_current, title) in zip(axes, predictions):
    ax.scatter(y_test, y_pred_current, alpha=0.6)
    ax.plot([x_min, x_max], [x_min, x_max], 'r--', label="Ideal Fit")
    ax.set_xlabel("Actual Complication Count")
    ax.set_ylabel("Predicted Complication Count")
    ax.set_title(title)
    ax.legend()

plt.tight_layout()
plt.show()



In [ ]:
# saving the model for deployment

import pickle

# Assuming 'model' is your trained model
with open("..\\models\\poisson_reg_model.pkl", "wb") as f:
    pickle.dump(rf, f)
